# Lab 5 · Renal Dose Adjustment Helper

We will estimate a recommended maintenance dose based on renal function, age, and a target plasma concentration.

### Learning goals
- Combine renal function metrics with pharmacodynamic goals.
- Train a linear regression model with three intuitive predictors.
- Evaluate the mean absolute dose error.
- Practice creating dose simulations for new patients.

In [ ]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

data_path = Path('..') / 'data' / 'renal_dose_adjustment.csv'
df = pd.read_csv(data_path)
print(f"Loaded {len(df)} patients.")
df.head()

### Exercise 1 · Summaries

In [ ]:
df.describe()

### Exercise 2 · Fit the model

In [ ]:
feature_cols = [
    'creatinine_clearance_ml_min',
    'age_years',
    'target_concentration_mcg_ml'
]
X = df[feature_cols]
y = df['recommended_dose_mg']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=11
)

model = LinearRegression()
model.fit(X_train, y_train)

print(f"Intercept: {model.intercept_:.2f} mg")
for name, coef in zip(feature_cols, model.coef_):
    print(f"{name}: {coef:.2f} mg per unit")

### Exercise 3 · Evaluate and simulate

In [ ]:
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.2f} mg")

results = X_test.copy()
results['actual_dose_mg'] = y_test.values
results['predicted_dose_mg'] = y_pred.round(1)
results

### Exercise 4 · New patient scenario

In [ ]:
new_patient = pd.DataFrame(
    {
        'creatinine_clearance_ml_min': [60],
        'age_years': [67],
        'target_concentration_mcg_ml': [2.1]
    },
    index=['Patient_X']
)

predicted_dose = model.predict(new_patient)[0]
print(f"Recommended dose: {predicted_dose:.1f} mg")
new_patient['recommended_dose_mg'] = predicted_dose
new_patient

### Reflection prompt
How would you explain this tool to a clinician with no coding background?